In [2]:
import os
import csv
import yaml

In [4]:
with open("./../config.yaml") as f:
    cfg = yaml.safe_load(f)
    
DATA_PATH = cfg["paths"]["waveform_dir"]

SAVE_FILE = "./../mimiciv_wfdb_meta.csv"

In [5]:
with open(SAVE_FILE, "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile,
        fieldnames=["subject_id","record_id","hadm_id"])
    writer.writeheader()

    for prefix in os.listdir(DATA_PATH):
        prefix_path = os.path.join(DATA_PATH, prefix)
        if not os.path.isdir(prefix_path) or not prefix.startswith("p"):
            continue

        for subj_dir in os.listdir(prefix_path):
            subj_path = os.path.join(prefix_path, subj_dir)
            if not os.path.isdir(subj_path) or not subj_dir.startswith(prefix):
                continue
            subject_id = subj_dir[1:] # delete the 'p' prefix
            
            for rec in os.listdir(subj_path):
                rec_path = os.path.join(subj_path, rec)
                hea_file = os.path.join(rec_path, rec + ".hea")
                if not os.path.isfile(hea_file):
                    continue
                
                hadm_id = None
                with open(hea_file) as f:
                    for line in f:
                        if line.startswith("# hadm_id"):
                            hadm_id = line.split()[2]
                            break
                if hadm_id is None:
                    continue
                writer.writerow({
                    "subject_id": subject_id,
                    "record_id": rec,
                    "hadm_id": hadm_id
                })